In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145118 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.17-0ubuntu2~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.17-0ubuntu2~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.17-0ubuntu2~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive # 此时colab中出现drive的文件夹，里面就是你的google drive的根目录文件

In [0]:
import os
os.chdir("drive/Colab_Notebooks") 

In [2]:
!pip install -q mrcnn

     |████████████████████████████████| 61kB 8.6MB/s 


In [0]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import h5py
from PIL import Image

In [0]:
class HelmetDataset(Dataset):
    # 加载数据集定义
    def load_dataset(self, dataset_dir, is_train=True):
        # 定义识别需要的两个类
        self.add_class("dataset", 1, "PersonWithHelmet")
        self.add_class("dataset", 2, "PersonWithoutHelmet")
        # 数据所在位置
        images_dir = dataset_dir + 'Selected_Revised/'
        annotations_dir = dataset_dir + 'Annotations/'
        # 定位到所有图像
        for filename in listdir(images_dir):
            if filename[4:5] == '2' and filename[0:1] == 'p':
                image_id = str(int(filename[-9:-4]) + 10000)
            elif filename[4:5] == 'A':
                image_id = str(int(filename[-9:-4]) + 20000)
            elif filename[4:5] == 'B':
                image_id = str(int(filename[-9:-4]) + 30000)
            else:
                image_id = filename[-9:-4]

            # 训练集略过 500 序号之前的所有图像
            if is_train and int(image_id) <= 900:
                continue
            # 测试集略过 500 序号之后的所有图像
            if not is_train and int(image_id) > 900:
                continue
            if int(image_id) == 72 or int(image_id) == 20535 or int(image_id) == 20748 or int(image_id) == 20414 or int(image_id) == 552:
                continue
            if int(image_id) == 20125 or int(image_id) == 20738 or int(image_id) == 20736 or int(image_id) == 20446 or int(image_id) == 20206:
                continue
            if int(image_id) == 20498 or int(image_id) == 20501 or int(image_id) == 20505 or int(image_id) == 20650 or int(image_id) == 990:
                continue
            if int(image_id) == 20183 or int(image_id) == 30105 or int(image_id) == 20220 or int(image_id) == 31493 or int(image_id) == 20476:
                continue
            if int(image_id) == 20762 or int(image_id) == 32271 or int(image_id) == 32287 or int(image_id) == 30233:
                continue
            if int(image_id) == 73 or int(image_id) == 74 or int(image_id) == 75 or int(image_id) == 32274:
                continue
            img_path = images_dir + filename
            ann_path = annotations_dir + filename[:-4] + '.xml'
            # 添加到数据集
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # 从注解文件中提取边框值
    def extract_boxes(self, filename):
        # 加载并解析文件
        tree = ElementTree.parse(filename)
        # 获取文档根元素
        root = tree.getroot()
        # 提取出图像尺寸及每个 bounding box 元素
        boxes = list()
        for object in root.findall('.//object'):
            name = object.find('name').text
            xmin = int(object.find('.//bndbox/xmin').text)
            ymin = int(object.find('.//bndbox/ymin').text)
            xmax = int(object.find('.//bndbox/xmax').text)
            ymax = int(object.find('.//bndbox/ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)

        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height

    # 加载图像掩膜
    def load_mask(self, image_id):
        # 获取图像详细信息
        info = self.image_info[image_id]
        # 定义盒文件位置
        path = info['annotation']
        # 加载 XML
        boxes, w, h = self.extract_boxes(path)
        # 为所有掩膜创建一个数组，每个数组都位于不同的通道
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        # 创建掩膜
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            if box[4] == 'hat':
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('PersonWithHelmet'))
            else:
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('PersonWithoutHelmet'))
        return masks, asarray(class_ids, dtype='int32')

    # 加载图像引用
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [31]:
# 训练集
train_set = HelmetDataset()
train_set.load_dataset('', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# 测试集
test_set = HelmetDataset()
test_set.load_dataset('', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Train: 4060
Test: 852


In [0]:
# 定义模型配置
class HelmetConfig(Config):
    # 定义配置名
    NAME = "Helmet_cfg"
    # 类的数量
    NUM_CLASSES = 1 + 2
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4
    IMAGE_MIN_DIM = 400
    IMAGE_MAX_DIM = 512
    TRAIN_ROIS_PER_IMAGE = 100
    # 每轮训练的迭代数量
    STEPS_PER_EPOCH = 1015
    VALIDATION_STEPS = 213

In [33]:
# 配置信息
config = HelmetConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     4
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 4
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  400
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES

In [0]:
# 定义模型
model = MaskRCNN(mode='training', model_dir='./', config=config)
model.keras_model.metrics_tensors = []
# 加载 mscoco 权重信息，排除输出层
model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

In [0]:
# 训练
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=4, layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: ./helmet_cfg20200307T0948/mask_rcnn_helmet_cfg_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (Tim

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multi

Epoch 1/4
  44/1015 [>.............................] - ETA: 1:41:12 - loss: 2.2882 - rpn_class_loss: 0.1027 - rpn_bbox_loss: 0.5459 - mrcnn_class_loss: 0.4709 - mrcnn_bbox_loss: 0.5863 - mrcnn_mask_loss: 0.5825